In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk, makedirs
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

from matplotlib import pyplot as plt

In [ ]:
from bulletin import default_input, default_output, tables_path, root

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash

from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

database_path = join(root,'database','leitos')

In [ ]:
default_output = join(default_output,'vacinacao',f'internados')
    
if not isdir(default_output): 
    makedirs(default_output)
    
default_output

In [ ]:
join_columns_levels = lambda df: df.set_axis([" ".join(map(str,columns)) for columns in df.columns], axis=1)

In [ ]:
situacao_contagio_labels = { -1: 'Não Vacinado / Esquema Primário Incompleto', 0:'Não Vacinado', 1:'Esquema Primário Incompleto', 2:'Esquema Primário Completo ', 3: 'Esquema Primário Completo + Reforço'}
fator_risco_labels = { 0:'Sem Fator de Risco', 1:'Com Fator de Risco' }
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}
evolucao_labels = {1: 'Recuperados', 2: 'Óbitos', 3: 'Ativos'}
raca_cor_labels = CasosConfirmados().tables['raca_cor'].set_index('id')['raca_cor'].to_dict()

In [ ]:
cc_vacinados = CasosConfirmados('cc_vacinados')
cc_vacinados.load()
casos = cc_vacinados.df

In [ ]:
casos.loc[casos['raca_cor'].isin([0,6]),'raca_cor'] = 99

In [ ]:
internamentos = pd.read_pickle(join(database_path,'internamentos.pkl')).sort_values('Cod Notificação COVID')
internamentos.shape

cc_vacinados = CasosConfirmados('cc_vacinados')
cc_vacinados.load()

internados = internamentos.drop_duplicates('Cod Notificação COVID',keep='last').copy()
internados.shape

internados = internados.join(cc_vacinados.df.set_index('id'), on='Cod Notificação COVID')
internados = internados.loc[internados['fator_risco'].notna()]
internados.shape

internados = internados.rename(columns={'Cod Notificação COVID':'id'})

internados['ano'] = internados['Data Internação'].dt.year
internados['mes'] = internados['Data Internação'].dt.month

In [ ]:
internados['fx'] = '60+'
internados.loc[(internados['Idade']>=12)&(internados['Idade']<60),'fx'] = '12-59'
internados.loc[(internados['Idade']<12),'fx'] = '0-11'

In [ ]:
internados = internados.rename(columns={'Cod Notificação COVID':'id'})

In [ ]:

internados['situacao_contagio'] = 0

internados.loc[
    (
        (internados['doses_aplicadas']=='1ª Dose') 
        &
        (internados['Data Internação'] >= internados['data_aplicacao_1a_dose'])
    ) 
    |
    (
        (internados['doses_aplicadas']=='1ª Dose,2ª Dose') 
        &
        (internados['Data Internação'] >= internados['data_aplicacao_1a_dose'])
        &
        (internados['Data Internação'] <= internados['data_imunizacao']  )
    ) 
    | 
    (
        (internados['doses_aplicadas']=='Única') 
        & 
        (internados['Data Internação'] >= internados['data_aplicacao_dose_unica'])
        &
        (internados['Data Internação'] <= internados['data_imunizacao']  )
    )
, 'situacao_contagio'] = 1


internados.loc[
    internados['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose',
            '1ª Dose,2ª Dose,Adicional',
            'Única',
            'Única,Adicional',
        ]
    ) 
    &
    (internados['Data Internação'] > internados['data_imunizacao']  )
    
, 'situacao_contagio'] = 2


internados.loc[
    internados['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose,Reforço',
            'Única,Reforço',
        ]
    ) 
    &
    (internados['Data Internação'] > internados['data_imunizacao'])
    &
    (internados['Data Internação'] <= internados['data_imunizacao_reforco'])

, 'situacao_contagio'] = 2


internados.loc[
    internados['doses_aplicadas'].isin(
        [
            '1ª Dose,2ª Dose,Reforço',
            'Única,Reforço',
        ]
    ) 
    &
    (internados['Data Internação'] > internados['data_imunizacao_reforco'])

, 'situacao_contagio'] = 3

In [ ]:
internados_mes_situacao_contagio = internados.groupby(['ano','mes','situacao_contagio'])[['id']].count().unstack('situacao_contagio').fillna(0).astype(int).droplevel(0,1)
internados_mes_situacao_contagio = internados_mes_situacao_contagio.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(index=mes_labels, level='mes')
internados_mes_situacao_contagio.to_csv(join(default_output,'internados_mes_situacao_contagio.csv'))
internados_mes_situacao_contagio

In [ ]:
internados_mes_situacao_contagio_fator_risco = internados.groupby(['ano','mes','situacao_contagio','fator_risco'])[['id']].count().unstack(['situacao_contagio','fator_risco']).fillna(0).astype(int).droplevel(0,1).sort_index(axis=1, level=0)
internados_mes_situacao_contagio_fator_risco = internados_mes_situacao_contagio_fator_risco.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(columns=fator_risco_labels, level='fator_risco').rename(index=mes_labels, level='mes')

internados_mes_situacao_contagio_fator_risco = join_columns_levels(internados_mes_situacao_contagio_fator_risco)
internados_mes_situacao_contagio_fator_risco.to_csv(join(default_output,'internados_mes_situacao_contagio_fator_risco.csv'))
internados_mes_situacao_contagio_fator_risco

In [ ]:
internados_mes_situacao_contagio_leito_tipo = internados.groupby(['ano','mes','situacao_contagio','Leito Tipo'])[['id']].count().unstack(['situacao_contagio','Leito Tipo']).fillna(0).astype(int).droplevel(0,1).sort_index(axis=1, level=0)
internados_mes_situacao_contagio_leito_tipo = internados_mes_situacao_contagio_leito_tipo.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(index=mes_labels, level='mes')

internados_mes_situacao_contagio_leito_tipo = join_columns_levels(internados_mes_situacao_contagio_leito_tipo)
internados_mes_situacao_contagio_leito_tipo.to_csv(join(default_output,'internados_mes_situacao_contagio_leito_tipo.csv'))
internados_mes_situacao_contagio_leito_tipo

In [ ]:
internados_mes_situacao_contagio_faixa_etaria = internados.groupby(['ano','mes','situacao_contagio','fx'])[['id']].count().unstack(['fx','situacao_contagio']).fillna(0).astype(int).droplevel(0,1).sort_index(axis=1, level=0)
internados_mes_situacao_contagio_faixa_etaria = internados_mes_situacao_contagio_faixa_etaria.reindex( pd.MultiIndex.from_product([ list(x) for x in  internados_mes_situacao_contagio_faixa_etaria.columns.levels ], names=internados_mes_situacao_contagio_faixa_etaria.columns.names),fill_value=0, axis=1)

internados_mes_situacao_contagio_faixa_etaria = internados_mes_situacao_contagio_faixa_etaria.rename(columns=situacao_contagio_labels, level='situacao_contagio').rename(index=mes_labels, level='mes')
internados_mes_situacao_contagio_faixa_etaria = join_columns_levels(internados_mes_situacao_contagio_faixa_etaria)
internados_mes_situacao_contagio_faixa_etaria.to_csv(join(default_output,'internados_mes_situacao_contagio_faixa_etaria.csv'))
internados_mes_situacao_contagio_faixa_etaria

In [ ]:
nt = Notifica()
fatores = nt.schema.loc[nt.schema['group_name']=='fator_risco','column']
fatores = [ f for f in fatores if f not in ['outras_morbidades','periodo_gestacao','gestante_alto_risco','gestante']]

In [ ]:
internados_fator_risco = internados.loc[(internados['fator_risco']==1)]

internados_fator_risco_situacao_contagio = {}

for situacao_contagio in sorted(internados_fator_risco['situacao_contagio'].unique()):
    fator_risco = internados_fator_risco.loc[internados_fator_risco['situacao_contagio']==situacao_contagio,fatores]
    n = len(fator_risco)
    fator_risco = fator_risco == 1
    fator_risco = fator_risco.sum(axis=0).to_frame(name='Quantidade')
    fator_risco['%'] = fator_risco['Quantidade'] / n
    # fator_risco['%'] = fator_risco['Quantidade'] / fator_risco['Quantidade'].sum()
    fator_risco = fator_risco.sort_values('%',ascending=False)
    fator_risco.loc['TOTAL DE ÓBITOS','Quantidade'] = n
    internados_fator_risco_situacao_contagio[situacao_contagio] = fator_risco
    
internados_fator_risco_situacao_contagio = pd.concat(internados_fator_risco_situacao_contagio,names=['situacao_contagio','fator_risco'],axis=1)
internados_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'%')] = (internados_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'Quantidade')] / internados_fator_risco_situacao_contagio.loc['TOTAL DE ÓBITOS',(slice(None),'Quantidade')].sum()).values
internados_fator_risco_situacao_contagio = internados_fator_risco_situacao_contagio.rename(columns=situacao_contagio_labels, level='situacao_contagio')
internados_fator_risco_situacao_contagio = internados_fator_risco_situacao_contagio.reorder_levels(['fator_risco','situacao_contagio'], axis=1)
internados_fator_risco_situacao_contagio = join_columns_levels(internados_fator_risco_situacao_contagio)
internados_fator_risco_situacao_contagio.to_csv(join(default_output,'internados_fator_risco_situacao_contagio.csv'), sep = ';', decimal=',', float_format='%.3f')
internados_fator_risco_situacao_contagio

In [ ]:
internados.loc[internados['Leito Tipo'].str.upper().str.contains('UTI'), 'Leito Tipo'] = 'UTI'
internados.loc[~internados['Leito Tipo'].str.upper().str.contains('UTI'), 'Leito Tipo'] = 'ENFERMARIA'

In [ ]:
internados_vacinados_tipo = internados.groupby(['situacao_contagio','Leito Tipo','fator_risco'])[['id']].count().unstack(['Leito Tipo','fator_risco']).fillna(0).droplevel(0,1)
internados_vacinados_tipo = internados_vacinados_tipo.rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns=fator_risco_labels, level='fator_risco')
internados_vacinados_tipo.index.name = ''
internados_vacinados_tipo.columns.name = ''
internados_vacinados_tipo_csv = internados_vacinados_tipo.set_axis([f"{l1} {l2}" for l1,l2 in internados_vacinados_tipo.columns], axis=1)
internados_vacinados_tipo_csv.to_csv(join(default_output,'internados_sem_alta_vacinados_tipo.csv'), sep = ';', decimal=',', float_format='%.3f')
internados_vacinados_tipo_csv

In [ ]:
internados_vacinados_tipo.groupby(axis=1, level='Leito Tipo').sum()

In [ ]:
internados_situacao_contagio = internados.groupby('situacao_contagio',dropna=False)[['id']].count().rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns={'id':'quantidade'})
internados_situacao_contagio.to_csv(join(default_output,'internados_situacao_contagio.csv'))
internados_situacao_contagio

In [ ]:
internados_situacao_contagio_fator_risco = internados.groupby(['situacao_contagio','fator_risco'])[['id']].count().unstack('fator_risco').fillna(0).astype(int).droplevel(0,1)
internados_situacao_contagio_fator_risco = internados_situacao_contagio_fator_risco.rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns=fator_risco_labels, level='fator_risco')
internados_situacao_contagio_fator_risco.to_csv(join(default_output,'internados_situacao_contagio_fator_risco.csv'))
internados_situacao_contagio_fator_risco

In [ ]:
internados_situacao_contagio_faixa_etaria = internados.groupby(['situacao_contagio','fx'])[['id']].count().unstack('fx').fillna(0).astype(int).droplevel(0,1)
internados_situacao_contagio_faixa_etaria = internados_situacao_contagio_faixa_etaria.rename(index=situacao_contagio_labels, level='situacao_contagio')
internados_situacao_contagio_faixa_etaria.to_csv(join(default_output,'internados_situacao_contagio_faixa_etaria.csv'))
internados_situacao_contagio_faixa_etaria

In [ ]:
internados_situacao_contagio_sexo = internados.groupby(['situacao_contagio','sexo'])[['id']].count().unstack('sexo').fillna(0).astype(int).droplevel(0,1)
internados_situacao_contagio_sexo = internados_situacao_contagio_sexo.rename(index=situacao_contagio_labels, level='situacao_contagio')
internados_situacao_contagio_sexo.to_csv(join(default_output,'internados_situacao_contagio_sexo.csv'))
internados_situacao_contagio_sexo

In [ ]:
internados_situacao_contagio_raca_cor = internados.groupby(['situacao_contagio','raca_cor'])[['id']].count().unstack('raca_cor').fillna(0).astype(int).droplevel(0,1)
internados_situacao_contagio_raca_cor = internados_situacao_contagio_raca_cor.rename(index=situacao_contagio_labels, level='situacao_contagio').rename(columns=raca_cor_labels, level='raca_cor')
internados_situacao_contagio_raca_cor.to_csv(join(default_output,'internados_situacao_contagio_raca_cor.csv'))
internados_situacao_contagio_raca_cor